In [1]:
import numpy as np
import pandas as pd
from collections import Counter

In [2]:
!pip install lightfm

  Running setup.py bdist_wheel for lightfm: started
  Running setup.py bdist_wheel for lightfm: finished with status 'error'
  Complete output from command C:\ProgramData\Anaconda3\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\DenAS\\AppData\\Local\\Temp\\pip-build-q3p35lry\\lightfm\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d C:\Users\DenAS\AppData\Local\Temp\tmpm6s6d73zpip-wheel- --python-tag cp36:
  Compiling without OpenMP support.
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.6
  creating build\lib.win-amd64-3.6\lightfm
  copying lightfm\cross_validation.py -> build\lib.win-amd64-3.6\lightfm
  copying lightfm\evaluation.py -> build\lib.win-amd64-3.6\lightfm
  copying lightfm\lightfm.py -> build\lib.win-amd64-3.6\lightfm
  copying lightfm\_lightfm_fast.py -> build\lib.win-amd64-3.6\lightfm
  c

  Failed building wheel for lightfm
  Failed cleaning build dir for lightfm
Command "C:\ProgramData\Anaconda3\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\DenAS\\AppData\\Local\\Temp\\pip-build-q3p35lry\\lightfm\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record C:\Users\DenAS\AppData\Local\Temp\pip-r5dw57sk-record\install-record.txt --single-version-externally-managed --compile" failed with error code 1 in C:\Users\DenAS\AppData\Local\Temp\pip-build-q3p35lry\lightfm\


In [2]:
train = pd.read_csv('titanic/train.csv')
test = pd.read_csv('titanic/test.csv')

In [3]:
#train.head()

In [4]:
#test.head()

In [5]:
y_train = train.Survived
train.drop('Survived', axis=1, inplace=True)

In [6]:
train.columns == test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True], dtype=bool)

In [7]:
train['is_test'] = 0
test['is_test'] = 1

In [8]:
df = pd.concat([train, test])
#df = df.dropna()

In [9]:
df["isMale"] = df.Sex.replace({"male": 1, "female":0})
df.drop(["Sex", "Cabin", "Ticket", "Name", "PassengerId"], axis=1, inplace=True)

In [10]:
def age_class(row):
    if row['Age'] <= 10:
        return 'kind'
    elif ( row['Age'] > 10 ) and ( row['Age'] <= 17 ):
        return 'teenager'
    elif ( row['Age'] > 17 ) and ( row['Age'] <= 21 ):
        return 'youth'
    elif ( row['Age'] > 21 ) and ( row['Age'] <= 55 ):
        return 'mature'    
    elif ( row['Age'] > 55 ) and ( row['Age'] <= 75 ):
        return 'elderly'
    elif ( row['Age'] > 75 ):
        return 'old'    

In [11]:
df['age_class'] = df.apply( age_class, axis = 1 )

In [12]:
df_dummies = pd.get_dummies(df, columns=['Pclass', 'Embarked','age_class'])

In [13]:
#df_dummies.head(10)

In [14]:
#df_dummies.isnull().sum()

In [15]:
X_train = df_dummies[df_dummies.is_test==0].drop('is_test', axis=1)
X_test = df_dummies[df_dummies.is_test==1].drop('is_test', axis=1)

In [16]:
columns = X_train.columns

In [17]:
#X_train.head(10)

In [18]:
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [19]:
from sklearn.preprocessing import Imputer

In [20]:
imputer = Imputer(missing_values='NaN', strategy='mean', axis=0, verbose=0, copy=True)

In [21]:
imputer.fit(X_train)

Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)

In [22]:
X_train_imputed = imputer.transform(X_train)
X_train_imputed = pd.DataFrame(X_train_imputed, columns=columns)

In [23]:
#X_train_imputed.head(10)

In [24]:
from sklearn.preprocessing import StandardScaler

In [25]:
scaler = StandardScaler()

In [26]:
scaler.fit(X_train_imputed)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [27]:
X_train_imputed_scaled = scaler.transform(X_train_imputed)
X_train_imputed_scaled = pd.DataFrame(X_train_imputed_scaled, columns=columns)

In [28]:
#X_train_imputed_scaled.head(10)

In [29]:
X_test_imputed_scaled = scaler.transform(imputer.transform(X_test))

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
#X_train_imputed_scaled[14:188]

In [32]:
#y_train

In [33]:
X_train_fin, X_val, y_train_fin, y_val = train_test_split(X_train_imputed_scaled, y_train, test_size=0.2)

In [34]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [35]:
cs = 10**np.linspace(-3,1,5)
#cs

In [36]:
grid = {'C': cs}#, 'max_features': features_num}
gridsearch = GridSearchCV(LogisticRegression(), grid, scoring='accuracy', cv=5)

In [37]:
%%time
gridsearch.fit(X_train_fin, y_train_fin)

Wall time: 240 ms


GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': array([  1.00000e-03,   1.00000e-02,   1.00000e-01,   1.00000e+00,
         1.00000e+01])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=0)

In [38]:
sorted(gridsearch.grid_scores_, key = lambda x: -x.mean_validation_score)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:747: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.80478, std: 0.05071, params: {'C': 0.01},
 mean: 0.80478, std: 0.04857, params: {'C': 0.10000000000000001},
 mean: 0.80478, std: 0.04302, params: {'C': 10.0},
 mean: 0.80337, std: 0.04563, params: {'C': 1.0},
 mean: 0.75702, std: 0.04609, params: {'C': 0.001}]

In [39]:
gridsearch.best_params_

{'C': 0.01}

In [40]:
best_C = gridsearch.best_params_["C"]

In [41]:
best_C

0.01

In [42]:
from sklearn.metrics import accuracy_score

In [43]:
clf = LogisticRegression(C=best_C)

In [44]:
clf.fit(X_train_fin, y_train_fin)

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [45]:
y_val_pred = clf.predict(X_val)

In [46]:
accuracy_score(y_val, y_val_pred)

0.82122905027932958

In [47]:
clf.fit(X_train_imputed_scaled, y_train)

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [48]:
clf.predict_proba(X_test_imputed_scaled)[:10]

array([[ 0.7511061 ,  0.2488939 ],
       [ 0.55352062,  0.44647938],
       [ 0.74172481,  0.25827519],
       [ 0.79612608,  0.20387392],
       [ 0.49032499,  0.50967501],
       [ 0.75515036,  0.24484964],
       [ 0.38889586,  0.61110414],
       [ 0.67306348,  0.32693652],
       [ 0.40647085,  0.59352915],
       [ 0.85862841,  0.14137159]])

In [49]:
predictions = clf.predict(X_test_imputed_scaled)
predictions

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0,

In [50]:
submussion = 'PassengerId,Survived\n'
submussion += "\n".join(["{},{}".format(pid, prediction) for pid, prediction in zip(test.PassengerId, predictions)])

In [51]:
#submussion

In [52]:
with open('submission.txt', 'w') as file:
    file.write(submussion)

In [53]:
for col, val in zip(X_train.columns, clf.coef_[0]):
    print("{:30} {:.2f}".format(col, val))

Age                            -0.14
SibSp                          -0.15
Parch                          -0.03
Fare                           0.16
isMale                         -0.72
Pclass_1                       0.25
Pclass_2                       0.08
Pclass_3                       -0.27
Embarked_C                     0.08
Embarked_Q                     0.03
Embarked_S                     -0.10
age_class_elderly              -0.06
age_class_kind                 0.18
age_class_mature               0.04
age_class_old                  0.07
age_class_teenager             0.04
age_class_youth                -0.06


# Строим дерево

In [71]:
from sklearn.tree import DecisionTreeClassifier

In [72]:
clf = DecisionTreeClassifier()

In [77]:
clf.fit(X_train_imputed_scaled, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [74]:
from sklearn.tree import export_graphviz

def get_tree_dot_view(clf, feature_names=None, class_names=None):
    print(export_graphviz(clf, out_file=None, filled=True, feature_names=feature_names, class_names=class_names))

In [78]:
clf = DecisionTreeClassifier(max_depth=3)
clf.fit(X_train_imputed_scaled, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [85]:
txt = get_tree_dot_view(clf, list(X_train_imputed.columns), X_train_imputed.columns)

digraph Tree {
node [shape=box, style="filled", color="black"] ;
0 [label="isMale <= -0.309\ngini = 0.473\nsamples = 891\nvalue = [549, 342]\nclass = Age", fillcolor="#e5813960"] ;
1 [label="Pclass_3 <= -0.103\ngini = 0.383\nsamples = 314\nvalue = [81, 233]\nclass = SibSp", fillcolor="#399de5a6"] ;
0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;
2 [label="Age <= -2.093\ngini = 0.1\nsamples = 170\nvalue = [9, 161]\nclass = SibSp", fillcolor="#399de5f1"] ;
1 -> 2 ;
3 [label="gini = 0.5\nsamples = 2\nvalue = [1, 1]\nclass = Age", fillcolor="#e5813900"] ;
2 -> 3 ;
4 [label="gini = 0.091\nsamples = 168\nvalue = [8, 160]\nclass = SibSp", fillcolor="#399de5f2"] ;
2 -> 4 ;
5 [label="Fare <= -0.178\ngini = 0.5\nsamples = 144\nvalue = [72, 72]\nclass = Age", fillcolor="#e5813900"] ;
1 -> 5 ;
6 [label="gini = 0.484\nsamples = 117\nvalue = [48, 69]\nclass = SibSp", fillcolor="#399de54e"] ;
5 -> 6 ;
7 [label="gini = 0.198\nsamples = 27\nvalue = [24, 3]\nclass = Age", fillcolor="#e5813

In [88]:
txt

In [80]:
#! pip install pydot

  Running setup.py bdist_wheel for pydot: started
  Running setup.py bdist_wheel for pydot: finished with status 'done'
  Stored in directory: C:\Users\DenAS\AppData\Local\pip\Cache\wheels\47\8c\c2\4ac7bd0219b4ce5e08d03dc3467014c7a94b2efecfc29df2b2
Successfully built pydot


In [86]:
import pydot 

In [90]:
pydot.graph_from_edges(clf)

TypeError: 'DecisionTreeClassifier' object is not iterable

In [84]:
#pydot.graph_from_dot_data(open('twopi2.gv.txt').read())

In [54]:
#№from sklearn.preprocessing import StandardScaler

In [26]:
#№scaler = StandardScaler()

In [27]:
#№scaler.fit(X_train_imputed)

In [28]:
#X_train_imputed_scaled = scaler.transform(X_train_imputed)
#X_train_imputed_scaled = pd.DataFrame(X_train_imputed_scaled, columns=columns)

In [29]:
#X_train_imputed_scaled.head(3)

In [96]:
#X_test_imputed_scaled = scaler.transform(imputer.transform(X_test))

In [99]:
#X_test_imputed_scaled.head(3)

In [100]:
#from sklearn.decomposition import PCA

In [101]:
#pca = PCA(n_components=13)

In [102]:
#ppl = pca.fit_transform(X_train_imputed_scaled)

In [30]:
#pca.explained_variance_ratio_.sum()

In [31]:
#plt.plot(ppl[:,0], ppl[:,1],ppl[:,2],ppl[:,3], ppl[:,4], ppl[:,5], 'ro', alpha=0.1)
#plt.title('Пассажиры Титаника')

In [105]:
#from sklearn.cluster import KMeans

In [106]:
#n_clusters = 5

In [107]:
#kmeans = KMeans(n_clusters=n_clusters)

In [32]:
#kmeans.fit(X_train_imputed_scaled)

In [33]:
#cluster_labels = kmeans.predict(X_train_imputed_scaled)

In [34]:
#plt.title('Пассажиры Титаника')
#for i,color in zip(range(n_clusters),{'aqua', 'blue', 'fuchsia', 'green', 'maroon', 'orange',
#                  'pink', 'purple', 'red','yellow', 'violet', 'indigo', 'chartreuse', 'lime'}):
#    t = ppl[cluster_labels==i]
#    plt.plot(t[:,0], t[:,1], 'ro', alpha=0.1, c=color)

In [35]:
#X_train_imputed_scaled

In [36]:
#y_train

In [37]:
#from sklearn.model_selection import train_test_split

In [38]:
#X_train_fin, X_val, y_train_fin, y_val = train_test_split(X_train_imputed_scaled, y_train, test_size=0.2)